### Google Cloud Vision - Face Detection API ###

In [77]:
#!/usr/bin/env python

# Copyright 2015 Google, Inc
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Draws squares around faces in the given image."""
import argparse
import base64

from PIL import Image
from PIL import ImageDraw

from googleapiclient import discovery
import httplib2
from oauth2client.client import GoogleCredentials
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './auth.json'


# [START get_vision_service]
DISCOVERY_URL='https://vision.googleapis.com/$discovery/rest?version=v1'


def get_vision_service():
    credentials = GoogleCredentials.get_application_default()
    return discovery.build('vision', 'v1', credentials=credentials,
                           discoveryServiceUrl=DISCOVERY_URL)
# [END get_vision_service]


# [START detect_face]
def detect_face(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.
    Args:
        face_file: A file-like object containing an image with faces.
    Returns:
        An array of dicts with information about the faces in the picture.
    """
    image_content = face_file.read()
    batch_request = [{
        'image': {
            'content': base64.b64encode(image_content).decode('UTF-8')
            },
        'features': [{
            'type': 'FACE_DETECTION',
            'maxResults': max_results,
            }]
        }]

    service = get_vision_service()
    request = service.images().annotate(body={
        'requests': batch_request,
        })
    response = request.execute()
    return response['responses'][0]['faceAnnotations']
# [END detect_face]

# [START print_landmarks]
def print_landmarks(face, image):
    """prints out landmark results for a face object"""
    coordinates = list()
    width = image.size[0]
    height = image.size[1]
    x_norm = y_norm = float(250); #relatively arbitrary, right now just approximated by cropped face size from ck dataset
    x_norm_ratio = x_norm/width;
    y_norm_ratio = y_norm/height;
    for landmark in face['landmarks']:
        # print landmark['type']
        # print 'x: ' + str(landmark['position']['x'])
        # print 'y: ' + str(landmark['position']['y'])
        # print 'z: ' + str(landmark['position']['z'])
        coordinates.append((landmark['position']['x'])*x_norm_ratio)
        coordinates.append((landmark['position']['y'])*y_norm_ratio)
        coordinates.append(landmark['position']['z'])
    return coordinates

    #print face['landmarks'][1]
# [END print_landmarks]

# def normalize_landmark(width, height, x_coord, y_coord, x_norm, y_norm): #norm - 250, 250
#     #return ()
#     return 0

# [START crop_face]
def crop_face(image, face, output_filename):
    """Crops a polygon around the faces, then saves to output_filename.
    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the faces
          have polygons drawn around them.
    """
    im = Image.open(image)

    box = [(v.get('x', 0.0), v.get('y', 0.0)) for v in face['fdBoundingPoly']['vertices']]
    corners = box[0] + box[2]
    im = im.crop(corners)
    im.save('images/' + output_filename)
    return im
# [END crop_face]

# [START face_detection]
def face_detection(input_filename, output_filename, max_results):
    with open(input_filename, 'rb') as image:
        face = detect_face(image, 1)[0] #detects face in original image <- inefficient way to find crop region?
        face_crop = crop_face(image, face, output_filename) #crop original image to just facial region
        with open('images/' + output_filename, 'rb') as image_crop: #rerun facial detection on cropped face
            face = detect_face(image_crop, 1)[0]
        return print_landmarks(face, Image.open('images/'+output_filename))
        #print('Writing to file %s' % output_filename)
        # Reset the file pointer, so we can read the file again
        #image.seek(0)
        #highlight_faces(image, faces, output_filename)
# [END face_detection]


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(
#         description='Detects faces in the given image.')
#     parser.add_argument(
#         'input_image', help='the image you\'d like to detect faces in.')
#     parser.add_argument(
#         '--out', dest='output', default='out.jpg',
#         help='the name of the output file.')
#     parser.add_argument(
#         '--max-results', dest='max_results', default=4,
#         help='the max results of face detection.')
#     args = parser.parse_args()

#     main(args.input_image, args.output, args.max_results)


In [78]:
#test run
face_detection('./images/CK+/cohn-kanade-plus-images/S051/002/S051_002_00000010.png', "out.jpg", 1)

[75.81367015209126,
 83.23715874524714,
 -0.00054242183,
 180.04882129277567,
 78.53305703422053,
 14.456583,
 35.73074619771864,
 59.44246958174905,
 9.4364767,
 103.66621673003803,
 53.22846863117871,
 -14.290893,
 156.07673954372626,
 51.174287072243345,
 -7.1145382,
 216.32480988593156,
 53.97913403041825,
 33.93354,
 131.37735741444868,
 74.72743916349809,
 -15.821347,
 137.98660646387833,
 131.50543726235742,
 -58.335003,
 138.3024049429658,
 174.43837452471485,
 -38.048069,
 138.9680133079848,
 212.615427756654,
 -35.473053,
 93.38339733840304,
 198.56468631178709,
 -16.282583,
 177.31248098859314,
 194.01611216730038,
 -4.5451975,
 137.39018060836503,
 191.52877376425855,
 -32.174747,
 166.0429657794677,
 144.23876425855514,
 -11.593722,
 105.35092205323194,
 147.40304182509507,
 -20.421896,
 137.48161596958175,
 151.84475285171104,
 -35.026592,
 75.17877756653994,
 74.9302633079848,
 -6.5142503,
 98.85173954372624,
 84.36564163498099,
 3.1294875,
 74.64830228136881,
 92.174515

In [79]:
#!/usr/bin/python

import os
from os import listdir
from cd import cd

def get_all_folders(folder_path):
    with cd(folder_path):
        all_folders = os.listdir('.')
        directory_names = []
        for directory in all_folders:
            if directory[0] == '.':
                continue
            else:
                directory_names.append(directory)
        return directory_names
    
labels = list()
features = list()

def grab_data(Folder):
    counter = 0
    
    ck_path = os.getcwd() + '/images/CK+/' + Folder + '/'
    s_folders = get_all_folders(ck_path)
    for folder in s_folders:
        subdir_path = ck_path + folder + '/'
        zero_folders = get_all_folders(subdir_path)
        for zero in zero_folders:
            img_level_path = subdir_path + zero + '/'
            imgs = get_all_folders(img_level_path)
            if Folder == 'Emotions': #LABELS
                if len(imgs) > 0: #has an emotion label
                    last_img_path = img_level_path + imgs[-1]
                    f = open(last_img_path, 'r')
                    emotion = f.read()
                    labels.append(emotion[3]) #emotion[3] is the number denoting emotion
                else: #has no emotion label
                    labels.append(-1)
            else: #FEATURES
                last_img_path = img_level_path + imgs[-1]
                if counter < len(labels): #TODO: CHECK THIS OFF BY ONE BULLSHIT
                    if labels[counter] != -1:
                        print counter
                        features.append(face_detection(last_img_path, "out.jpg", 1))
                counter += 1
#                 if counter > 5: #because of bad internet
#                     return

grab_data('Emotions')
grab_data('cohn-kanade-plus-images')
temp_labels = list()
for label in labels: #clean temp labels
    if label != -1:
        temp_labels.append(label)
labels = temp_labels


0
2
4
6
7
8
9
10
11
12
13
14
15
17
18
22
23
24
25
26
29
30
31
32
34
35
36
37
38
40
42
43
47
48
49
51
54
55
56
58
60
61
63
66
70
71
72
73
74
75
77
78
80
82
84
85
86
89
90
91
93
95
96
97
98
100
101
102
103
105
106
107
108
110
113
114
117
118
119
120
123
124
125
126
127
128
129
130
132
133
134
135
136
138
139
141
142
143
144
146
147
148
149
151
153
154
155
157
158
159
160
162
163
164
167
168
169
170
171
173
174
175
180
181
182
187
188
189
191
192
194
197
198
200
201
205
206
207
212
214
217
220
221
222
224
225
229
230
231
232
236
237
241
243
246
248
249
251
252
253
254
255
256
259
260
262
263
266
267
268
269
270
271
272
273
275
276
278
279
280
283
284
286
287
292
293
294
295
297
298
299
302
304
307
308
309
312
313
314
317
318
320
325
326
332
349
351
353
355
357
361
366
367
369
372
374
375
376
383
388
389
394
395
397
402
403
408
409
412
416
417
422
423
424
426
429
437
439
444
452
456
458
461
462
463
467
468
469
470
471
472
473
476
477
479
480
481
482
484
485
486
492
494
495
496
497
499
502


In [94]:
## CONVERT TO FLOATS ##
import numpy as np

labels = np.array(labels).astype(np.float) #does it matter if the labels are strings or floats? what's the difference?
#print labels

features = np.array(features).astype(np.float)
print features

[[  7.49039760e+01   8.30994489e+01   1.55015050e-03 ...,   2.30113339e+02
    1.83858450e+02   1.13633720e+02]
 [  7.10013750e+01   8.52327887e+01  -1.73937260e-03 ...,   2.40550000e+02
    2.07707702e+02   9.99086380e+01]
 [  6.97876577e+01   8.13421929e+01  -9.44529950e-04 ...,   2.36836720e+02
    1.89456563e+02   9.12832720e+01]
 ..., 
 [  6.05348421e+01   7.99952588e+01   2.10058920e-03 ...,   2.17467281e+02
    1.93193500e+02   9.32072830e+01]
 [  7.25139441e+01   8.31724095e+01  -1.86232990e-03 ...,   2.30141464e+02
    2.02685329e+02   8.96114810e+01]
 [  7.53539794e+01   8.68704841e+01   1.41212260e-03 ...,   2.37296381e+02
    2.11071754e+02   6.83160550e+01]]


In [95]:
# import os
# if not os.path.exists('my_file'): numpy.savetxt('my_file', my_array)

np.savetxt("ck_features_norm", features)
np.savetxt("ck_labels", labels)
np.loadtxt("ck_labels")

array([ 3.,  7.,  1.,  5.,  7.,  6.,  4.,  1.,  3.,  5.,  7.,  6.,  1.,
        5.,  7.,  1.,  3.,  7.,  6.,  1.,  5.,  1.,  1.,  7.,  1.,  4.,
        3.,  5.,  7.,  1.,  5.,  7.,  3.,  5.,  7.,  1.,  5.,  7.,  6.,
        1.,  5.,  7.,  5.,  3.,  3.,  1.,  6.,  7.,  4.,  3.,  4.,  7.,
        1.,  5.,  7.,  3.,  7.,  5.,  3.,  7.,  5.,  4.,  7.,  3.,  7.,
        3.,  1.,  5.,  4.,  3.,  7.,  5.,  7.,  3.,  5.,  7.,  1.,  3.,
        7.,  4.,  5.,  7.,  3.,  7.,  5.,  3.,  4.,  7.,  5.,  3.,  7.,
        5.,  7.,  5.,  6.,  4.,  7.,  5.,  3.,  7.,  5.,  6.,  1.,  7.,
        1.,  5.,  3.,  5.,  7.,  4.,  3.,  7.,  3.,  5.,  7.,  5.,  3.,
        7.,  6.,  1.,  5.,  3.,  1.,  5.,  7.,  3.,  4.,  7.,  3.,  5.,
        7.,  3.,  5.,  1.,  7.,  3.,  5.,  7.,  3.,  7.,  5.,  3.,  7.,
        3.,  5.,  7.,  6.,  3.,  7.,  6.,  3.,  7.,  1.,  3.,  5.,  7.,
        4.,  5.,  7.,  3.,  7.,  5.,  7.,  3.,  5.,  1.,  7.,  3.,  7.,
        5.,  1.,  7.,  3.,  1.,  4.,  5.,  7.,  1.,  5.,  6.,  5

In [96]:
# you can now use np.loadtxt("") to load in some of the google api generated landmark arrays.
np.loadtxt("ck_features_norm")

array([[  7.49039760e+01,   8.30994489e+01,   1.55015050e-03, ...,
          2.30113339e+02,   1.83858450e+02,   1.13633720e+02],
       [  7.10013750e+01,   8.52327887e+01,  -1.73937260e-03, ...,
          2.40550000e+02,   2.07707702e+02,   9.99086380e+01],
       [  6.97876577e+01,   8.13421929e+01,  -9.44529950e-04, ...,
          2.36836720e+02,   1.89456563e+02,   9.12832720e+01],
       ..., 
       [  6.05348421e+01,   7.99952588e+01,   2.10058920e-03, ...,
          2.17467281e+02,   1.93193500e+02,   9.32072830e+01],
       [  7.25139441e+01,   8.31724095e+01,  -1.86232990e-03, ...,
          2.30141464e+02,   2.02685329e+02,   8.96114810e+01],
       [  7.53539794e+01,   8.68704841e+01,   1.41212260e-03, ...,
          2.37296381e+02,   2.11071754e+02,   6.83160550e+01]])

In [97]:
### ARRAY LENGTH SANITY CHECK ###
print len(features)
print len(labels)


327
327


In [98]:
#tools
def train_test_split(split): 
    features_train = features[:split]
    labels_train = labels[:split]
    features_test = features[split:]
    labels_test = labels[split:]
    return (features_train, labels_train, features_test, labels_test)

def num_of_label(val, labels):
    count = 0
    for label in labels:
        if label == val:
            count += 1
    return count

def print_accuracy(labels_actual, labels_predicted):
    correct = 0
    wrong = 0
    for index in range(len(labels_actual)):
        if labels_actual[index] == labels_predicted[index]:
            correct += 1
        else:
            wrong += 1
    print 'num correct: ' + str(correct)
    print 'num wrong: ' + str(wrong)
    print 'accuracy: ' + str(float(correct)/(correct + wrong))

In [103]:
from sklearn import svm

#X is a matrix of input examples [[x1.....xn],...[x1,..., xn]]
#Y is a vector of labels corresponding to emotion [1, 2, 8, ... yn]


def train(X, y):
#    clf = svm.SVC(decision_function_shape='ovo')
    clf = svm.SVC(kernel='linear', C=1)
#     clf = svm.LinearSVC()
    clf.fit(X,y)
    print 'Finished Training SVM'
    return clf

def test(clf, X):
    #Takes all examples to predict and returns an array of lables
    predictions = clf.predict(X)
    print 'Finished Predictions for SVM'
    return predictions

############ Sample usage ####################

#X = [[0,1], [3,4], [6,2], [1,2]]
#y = [1,2,3,4]
    
splits = train_test_split(270)
features_train = splits[0]
labels_train = splits[1]
features_test = splits[2]
labels_test = splits[3]

clf = train(features_train, labels_train)
predictions = test(clf, features_test)

# print(clf.score(features_test, labels_test))

print "training labels: "
print labels_train

for val in range (0, 8):
    print "TRAIN: num of label " + str(val) + ": " + str(num_of_label(val, labels_train))

### ERROR ANALYSIS ### 
print "predictions: "
print predictions
print "actual: "
print labels_test

print_accuracy(labels_test, predictions)

Finished Training SVM
Finished Predictions for SVM
training labels: 
[ 3.  7.  1.  5.  7.  6.  4.  1.  3.  5.  7.  6.  1.  5.  7.  1.  3.  7.
  6.  1.  5.  1.  1.  7.  1.  4.  3.  5.  7.  1.  5.  7.  3.  5.  7.  1.
  5.  7.  6.  1.  5.  7.  5.  3.  3.  1.  6.  7.  4.  3.  4.  7.  1.  5.
  7.  3.  7.  5.  3.  7.  5.  4.  7.  3.  7.  3.  1.  5.  4.  3.  7.  5.
  7.  3.  5.  7.  1.  3.  7.  4.  5.  7.  3.  7.  5.  3.  4.  7.  5.  3.
  7.  5.  7.  5.  6.  4.  7.  5.  3.  7.  5.  6.  1.  7.  1.  5.  3.  5.
  7.  4.  3.  7.  3.  5.  7.  5.  3.  7.  6.  1.  5.  3.  1.  5.  7.  3.
  4.  7.  3.  5.  7.  3.  5.  1.  7.  3.  5.  7.  3.  7.  5.  3.  7.  3.
  5.  7.  6.  3.  7.  6.  3.  7.  1.  3.  5.  7.  4.  5.  7.  3.  7.  5.
  7.  3.  5.  1.  7.  3.  7.  5.  1.  7.  3.  1.  4.  5.  7.  1.  5.  6.
  5.  7.  5.  7.  3.  5.  6.  7.  3.  5.  7.  3.  5.  3.  5.  7.  5.  3.
  7.  1.  5.  7.  7.  4.  3.  3.  6.  3.  5.  7.  3.  6.  3.  5.  1.  3.
  5.  7.  7.  1.  3.  1.  7.  6.  1.  7.  5.  7.  6.  5

In [104]:
##LINEAR REGRESSION##
from sklearn import linear_model
#clf = linear_model.Ridge (alpha = .5)
clf = linear_model.LinearRegression()
clf.fit(features_train, labels_train)
# print("coefficient: ")
# clf.coef_
# print("predict: ")
test_results = clf.predict(features_test)
updated_results = list()
for result in test_results:
    updated_results.append(int(round(result)))
updated_results = np.array(updated_results).astype(np.float)
print updated_results
print labels_test

print_accuracy(labels_test, updated_results)


[  5.   5.   6.   6.   6.   4.   6.   8.   6.   7.   3.   7.   5.   5.   3.
   5.   8.  10.   4.   0.   1.   3.   7.   8.   9.   9.   7.   4.   6.   7.
   6.   5.   5.   6.   5.   4.   3.   4.   6.   5.   4.   7.   3.   4.   3.
   4.   5.   6.   4.   8.   8.   7.   6.   5.   7.   7.   4.]
[ 4.  3.  5.  7.  1.  7.  5.  1.  5.  3.  7.  5.  7.  6.  1.  5.  7.  6.
  5.  4.  7.  5.  6.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  1.
  4.  6.  1.  2.  4.  1.  2.  6.  1.  2.  4.  6.  2.  6.  1.  2.  4.  6.
  2.  1.  4.]
num correct: 4
num wrong: 53
accuracy: 0.0701754385965


In [149]:
##RANDOM##
import random

num_examples = len(predictions)
random_labels = [random.randint(0,7) for p in range(0,num_examples)]
# print random_labels
# print labels_test
print_accuracy(labels_test, random_labels)

num correct: 17
num wrong: 110
accuracy: 0.133858267717


In [101]:
### CROSS VALIDATION ###
from sklearn import metrics
from sklearn import cross_validation

clf = svm.SVC(kernel='linear', C=1)
scores = cross_validation.cross_val_score(clf, features, labels, cv=5)

scores                                              


array([ 0.35820896,  0.35820896,  0.28358209,  0.125     ,  0.11290323])